In [1]:
%reset -f
import glob, os, sys, io
sys.path.insert(1, os.path.join(sys.path[0], '..'))
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing
from sklearn.model_selection import train_test_split
import keras

from wavhandler import *
from utils import *

import logging
logger = logging.getLogger()
logger.propagate = False
logger.setLevel(logging.ERROR)
np.random.seed(0)
import seaborn as sns
sns.set()

Using TensorFlow backend.


In [2]:
X,y = get_data(target_names=all_6, nr_signals=20000, only_names=False)
#X = transform_data(X)

100%|██████████| 6/6 [00:15<00:00,  2.57s/it]


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    shuffle=True, 
                                                    random_state=0)

In [4]:
# Convert label to onehot
y_train = keras.utils.to_categorical(y_train, num_classes=6)
y_test = keras.utils.to_categorical(y_test, num_classes=6)

X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

# Build the Neural Network
model = Sequential()

model.add(Conv1D(16, 3, activation='relu', input_shape=(5000, 1)))
model.add(Conv1D(16, 3, activation='relu'))
model.add(BatchNormalization())

model.add(Conv1D(32, 3, activation='relu'))
model.add(Conv1D(32, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Conv1D(64, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(256, 3, activation='relu'))
model.add(Conv1D(256, 3, activation='relu'))
model.add(BatchNormalization())
model.add(GlobalAveragePooling1D())

model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


In [6]:
model_name = 'deep_1'
top_weights_path = './data/model_' + str(model_name) + '.h5'

In [7]:
callbacks_list = [ModelCheckpoint(top_weights_path, monitor = 'val_acc', verbose = 1, save_best_only = True, save_weights_only = True),
    EarlyStopping(monitor = 'val_acc', patience = 6, verbose = 1),
    ReduceLROnPlateau(monitor = 'val_acc', factor = 0.1, patience = 3, verbose = 1),
    CSVLogger('model_' + str(model_name) + '.log')]

In [8]:
model.fit(X_train, y_train, batch_size=128, epochs=100, validation_data = [X_test, y_test], callbacks = callbacks_list)


Train on 96724 samples, validate on 24182 samples
Epoch 1/100
96724/96724 [==============================] - 600s 6ms/step - loss: 0.6198 - acc: 0.7673 - val_loss: 1.6741 - val_acc: 0.5633

Epoch 00001: val_acc improved from -inf to 0.56331, saving model to model_deep_1.h5
Epoch 2/100
96724/96724 [==============================] - 595s 6ms/step - loss: 0.4051 - acc: 0.8492 - val_loss: 1.8552 - val_acc: 0.5222

Epoch 00002: val_acc did not improve from 0.56331
Epoch 3/100
96724/96724 [==============================] - 593s 6ms/step - loss: 0.3529 - acc: 0.8680 - val_loss: 3.4001 - val_acc: 0.4790

Epoch 00003: val_acc did not improve from 0.56331
Epoch 4/100
96724/96724 [==============================] - 594s 6ms/step - loss: 0.3185 - acc: 0.8804 - val_loss: 9.7684 - val_acc: 0.2241

Epoch 00004: val_acc did not improve from 0.56331

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 5/100
96724/96724 [==============================] - 594s 6ms/step -

In [9]:
model.load_weights(top_weights_path)
loss, acc = model.evaluate(X_test, y_test, batch_size=16)

#print('loss', loss)
print('Test accuracy:', acc)

24182/24182 [==============================] - 49s 2ms/step
Test accuracy: 0.9134893722652886
